In [ ]:
!pip install langchain openai requests

In [ ]:
import openai
import requests
#from langchain import LangChain # This line is causing the error
from langchain.chains import LLMChain # Import LLMChain instead of LangChain
from langchain.prompts import PromptTemplate

# Set up your OpenAI API key
openai.api_key = 'your-api-key'

# Define email templates
email_templates = {
    "collaboration": """
Subject: {subject}

Hi {recipient_name},

I hope this email finds you well. My name is {sender_name} from {company_name}. I came across your profile and was impressed by your work in {industry}.

At {company_name}, we specialize in {product_service} that can help you with {value_proposition}. I believe this could be a great fit for your needs.

I'd love to discuss this further and see how we can collaborate. Please let me know a convenient time for a quick call.

Best regards,
{sender_name}
{sender_position}
{company_name}
""",
    "job opportunity": """
Subject: {subject}

Hi {recipient_name},

I hope this email finds you well. My name is {sender_name} from {company_name}. We are currently expanding our team and your experience in {industry} caught our attention.

At {company_name}, we are committed to {value_proposition}. We believe your expertise in {product_service} would be a valuable addition to our team.

I would love to discuss this opportunity with you further. Could we schedule a call at your earliest convenience?

Best regards,
{sender_name}
{sender_position}
{company_name}
""",
    "product inquiry": """
Subject: {subject}

Hi {recipient_name},

I hope this email finds you well. My name is {sender_name} from {company_name}. I noticed your interest in {industry} and thought you might be interested in our {product_service}.

At {company_name}, we strive to {value_proposition}. I believe our {product_service} could be beneficial for your needs.

I'd be happy to provide more details and answer any questions you may have. Please let me know a convenient time for a quick call.

Best regards,
{sender_name}
{sender_position}
{company_name}
"""
}

# Fetch a Wikipedia article
def fetch_wikipedia_article(url):
    response = requests.get(url)
    return response.text

# Summarize text using OpenAI
def get_summary(text, max_length):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=f"Summarize the following text in less than {max_length} words: {text}",
        max_tokens=max_length
    )
    return response.choices[0].text.strip()

# Get user input (placeholder function)
def get_user_input(prompt):
    return input(prompt)

# Generate email using template
def generate_email(data, template):
    email = template.format(
        subject=data['subject'],
        recipient_name=data['recipient_name'],
        sender_name=data['sender_name'],
        company_name=data['company_name'],
        industry=data['industry'],
        product_service=data['product_service'],
        value_proposition=data['value_proposition'],
        sender_position=data['sender_position']
    )
    return email

# Get personalization data based on the subject
# Get personalization data based on the subject
def get_personalization_data(subject):
    common_data = {
        "subject": get_user_input("Enter the email subject: "),
        "recipient_name": get_user_input("Enter the recipient's name: "),
        "sender_name": get_user_input("Enter your name: "),
        "company_name": get_user_input("Enter your company name: "),
        "sender_position": get_user_input("Enter your job title: ")
    }

    # Collect specific data based on the selected template,
    # using subject as fallback for default case
    if subject in email_templates:
        if subject == "collaboration":
            specific_data = {
                "industry": get_user_input("Enter the recipient's industry: "),
                "product_service": get_user_input("Enter the product/service you offer: "),
                "value_proposition": get_user_input("Enter the value proposition: ")
            }
        elif subject == "job opportunity":
            specific_data = {
                "industry": get_user_input("Enter the recipient's industry: "),
                "product_service": get_user_input("Enter the relevant expertise: "),
                "value_proposition": get_user_input("Enter the company value proposition: ")
            }
        elif subject == "product inquiry":
            specific_data = {
                "industry": get_user_input("Enter the recipient's industry: "),
                "product_service": get_user_input("Enter the product/service you offer: "),
                "value_proposition": get_user_input("Enter how the product/service benefits the recipient: ")
            }
        else:
            specific_data = {}
    else: # Fallback for subjects not matching any template
            specific_data = {
                "industry": get_user_input("Enter the recipient's industry: "),
                "product_service": get_user_input("Enter the product/service you offer: "),
                "value_proposition": get_user_input("Enter the value proposition: ")
            }

    return {**common_data, **specific_data}

# Define the email generation chain
class EmailGenerationChain:
    def __init__(self, template):
        self.template = template

    def run(self, data):
        email = generate_email(data, self.template)
        return email

# Main function
if __name__ == "__main__":
    # Ask user for email subject
    subject = get_user_input("Enter the email subject: ")

    # Select appropriate template based on the subject
    subject_keywords = {
        "collaboration": ["collaboration", "partner", "join"],
        "job opportunity": ["job", "opportunity", "hiring"],
        "product inquiry": ["product", "inquiry", "information"]
    }

    selected_template = None
    for key, keywords in subject_keywords.items():
        if any(keyword in subject.lower() for keyword in keywords):
            selected_template = email_templates[key]
            break
    if not selected_template:
        selected_template = email_templates["collaboration"]  # Default template

    # Ask user for personalization data based on the selected subject
    data = get_personalization_data(selected_template)

    # Create and run the email generation chain
    email_chain = EmailGenerationChain(template=selected_template)
    generated_email = email_chain.run(data)

    print(f"\nGenerated Email:\n{generated_email}")


Enter the email subject: job
Enter the email subject: job
Enter the recipient's name: ram
Enter your name: ravi
Enter your company name: presidency
Enter your job title: managetr
Enter the recipient's industry: software
Enter the product/service you offer: software application
Enter the value proposition: software

Generated Email:

Subject: job

Hi ram,

I hope this email finds you well. My name is ravi from presidency. We are currently expanding our team and your experience in software caught our attention.

At presidency, we are committed to software. We believe your expertise in software application would be a valuable addition to our team.

I would love to discuss this opportunity with you further. Could we schedule a call at your earliest convenience?

Best regards,
ravi
managetr
presidency

